### Edgar Moises Hernandez-Gonzalez
#### 20/04/20
#### Clasificacion de imagenes motoras en EEG con CNN-2D
#### Cargar modelo para evaluacion y predecir
##### Caracteristicas = Escalogramas CWT (3D)
##### Concatenacion vertical
##### Resize proporcional de 352x125

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import pywt
import cv2
from keras.models import load_model

Using TensorFlow backend.


In [0]:
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/MI-EEG-A01E.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RAW/EtiquetasA01E.csv", header=None)

In [0]:
print(x_test.shape)
print(y_test.shape)

(288, 22000)
(288, 1)


In [0]:
n_samples_test = len(y_test) #en caso de no tener el y_test se puede utilizar el x_test

print("n_samples_test :", n_samples_test)

n_samples_test : 288


In [0]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#codificar etiquetas con valores de 0 a nClases-1
y_test = LabelEncoder().fit_transform(y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
#unir 22 canales
def unir_escalogramas_vertical_resize(data):
  escalas = np.arange(1,129)
  dim = (125, 352) #ancho, alto
  datos = np.zeros((288,352,125))
  temporal = np.zeros((2816,1000))
  for i in range(data.shape[0]): #288 muestras
    for j in range(22): #22 canales
      sig = data.iloc[i, j*1000:(j+1)*1000]
      coef, freqs = pywt.cwt(sig, escalas, 'cmor3-3') #escalograma CWT
      temporal[j*128:(j+1)*128, :] = abs(coef)
    resized = cv2.resize(temporal, dim, interpolation=cv2.INTER_AREA)
    datos[i] = resized
    if i % 100 == 0: #esto solo es para ver como avanza
      print(i)
  return datos

In [0]:
x_test = unir_escalogramas_vertical_resize(x_test)

0
100
200


In [0]:
print(x_test.shape)

(288, 352, 125)


In [0]:
#convertir a float
x_test = x_test.astype('float32')

#escalar los valores en un rango de 0 a 1 (normalizar)
x_test /= 61.0

In [0]:
#convertir de 3D a 4D
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

print(x_test.shape)

(288, 352, 125, 1)


In [0]:
model = load_model('/content/drive/My Drive/BCI-IV-2a/Modelos/Mejor_modelo_CWTDA01.hdf5')

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

In [0]:
print("loss: ", test_loss)
print("accuracy: ", test_acc)

loss:  1.042798399925232
accuracy:  0.5138888955116272


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 350, 123, 8)       80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 175, 61, 8)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 173, 59, 8)        584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 86, 29, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 27, 8)         584       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 42, 13, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4368)             